In [48]:
!ls ../input/ffn-v2/ffn.pth

../input/ffn-v2/ffn.pth


In [49]:
!ls ../input/resnet200d-320-640-fold0-colab-enx7/resnet200d_320_640_fold0_colab_eNx7.pth

../input/resnet200d-320-640-fold0-colab-enx7/resnet200d_320_640_fold0_colab_eNx7.pth


In [50]:
!ls ../input/resnet200d-320-640-fold0-colab-enx2/resnet200d_320_fold0_colab_eNx2.pth

../input/resnet200d-320-640-fold0-colab-enx2/resnet200d_320_fold0_colab_eNx2.pth


In [51]:
!ls ../input/resnet200d-320-fold0-enx7/resnet200d_320_fold0_eNx7.pth

../input/resnet200d-320-fold0-enx7/resnet200d_320_fold0_eNx7.pth


In [52]:
!ls ../input/resnet200d320fold0-end/resnet200d_320_fold0_end.pth

../input/resnet200d320fold0-end/resnet200d_320_fold0_end.pth


In [53]:
import os
import sys
sys.path.append('../input/pytorch-images-seresnet')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import os
import gc
import time
import math
import random
import datetime
import pandas as pd
import numpy as np
import seaborn as sns

import torch
import torch.nn as nn
from   torch.nn import CrossEntropyLoss, MSELoss
from   torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from   torch.nn import Parameter
from   torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from   transformers import TrainingArguments, Trainer, AdamW, get_linear_schedule_with_warmup

from   fastai.losses import LabelSmoothingCrossEntropy

# from   warmup_scheduler import GradualWarmupScheduler
from   sklearn import preprocessing
from   sklearn.metrics import accuracy_score
from   sklearn.model_selection import StratifiedKFold, GroupKFold
from   sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef, roc_auc_score
from   sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

import timm
import albumentations as A
from   albumentations.pytorch import ToTensorV2
from   albumentations.core.transforms_interface import DualTransform
from   albumentations.augmentations import functional as AF
import cv2

from   tqdm import tqdm
from   pprint import pprint
from   functools import partial
import matplotlib.pyplot as plt
from   numba import cuda
import warnings
warnings.filterwarnings ("ignore")

In [54]:
class CFG:    
    IMAGE_SIZE = 640
    TEST_PATH  = '../input/ranzcr-clip-catheter-line-classification/test'
    device       = torch.device ('cuda' if torch.cuda.is_available () else 'cpu')
    num_workers  = 4
    model_name   = 'resnet200d_320' # ['deit_base_patch16_224', 'vit_base_patch16_384', 'resnext50_32x4d', 'tf_efficientnet_b7_ns']
    size         = 640  # [64, 128, 224, 384, 512, 640, 720]
    train        = True
    freeze       = True     # this is updated during training   
    freeze_epo   = 0.5       # float: these many epochs are with frozen model at the beginning
    epochs       = 1 
    epochsNx     = 40
    lr           = 5e-4
    criterion    = 'BCEWithLogitsLoss'    # ['CrossEntropyLoss', 'BCEWithLogitsLoss', 'SmoothBCEwithLogits']
    batch_size   = 95 #[10, 32, 64]
    weight_decay = 1e-6
    max_grad_norm= 1000.0
    seed         = 42
    target_size  = -1    # init below
    n_fold       = 50
    train_fold   = [0] #, 1, 2, 3, 4]
    print_every  = 100
    img_ext      = '.jpg'
    img_col      = "StudyInstanceUID"
    label_cols   = [
                    'ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                    'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                    'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                    'Swan Ganz Catheter Present'
    ]
    train_path   = '../input/ranzcr-clip-catheter-line-classification/train'
    train_csv    = '../input/ranzcr-clip-catheter-line-classification/train.csv'
    test_path    = '../input/ranzcr-clip-catheter-line-classification/test'
    test_csv     = '../input/ranzcr-clip-catheter-line-classification/sample_submission.csv'    
    
    MODEL_PATHS  = [
        '../input/resnet200d-320-640-fold0-colab-enx7/resnet200d_320_640_fold0_colab_eNx7.pth',
        '../input/resnet200d-320-640-fold0-colab-enx2/resnet200d_320_fold0_colab_eNx2.pth',
        '../input/resnet200d-320-fold0-enx7/resnet200d_320_fold0_eNx7.pth',
        '../input/resnet200d320fold0-end/resnet200d_320_fold0_end.pth',
    ]
    MODELS       = []
    MODEL_LOGITS = []
    BATCH_LABELS = []
    ffn_path     = '../input/ffn-v2/ffn.pth'

In [55]:
def seed_everything (seed):
    
    random.seed (seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed (seed)
    torch.manual_seed (seed)
    torch.cuda.manual_seed (seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    return

seed_everything (CFG.seed)

In [56]:
test_df = pd.read_csv (CFG.test_csv)

In [57]:
def get_transforms ():
        return A.Compose ([
            A.Resize (CFG.size, CFG.size),
            A.Normalize (),
            ToTensorV2 (),
        ])

In [58]:
class ImgDataset (Dataset):
    
    def __init__(self, df, img_file_colname=CFG.img_col, label_cols=CFG.label_cols, 
                 transform=get_transforms(), img_dir=CFG.train_path, img_ext=CFG.img_ext):
        
        super ().__init__()
        self.df               = df.reset_index (drop=True)
        self.img_ext          = CFG.img_ext
        self.img_dir          = img_dir
        self.label_cols       = label_cols
        self.img_file_colname = img_file_colname
        self.transform        = transform
        return
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        file_name = self.df[self.img_file_colname][idx].replace (self.img_ext, '') + self.img_ext
        file_path = f'{self.img_dir}/{file_name}'
        image     = cv2.imread (file_path)                              #;print (file_path)
        image     = cv2.cvtColor (image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform (image=image)['image'].float ()
        else:
            image = ToTensorV2 ()(image = image)["image"].float ()
        
        if len (self.label_cols) > 0:
            label = torch.tensor (self.df.loc[idx, self.label_cols]).float () # long ()
            return image, label
        return image

In [59]:
class ResNet200D_320(nn.Module):
    
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
    def freeze (self):
        # To freeze the residual layers
        for param in self.model.parameters ():
            param.requires_grad = False

        for param in self.fc.parameters ():
            param.requires_grad = False
        return

In [60]:
class ResNet200D (nn.Module):
    
    def __init__(self, model_name='resnet200d', out_dim=11, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, out_dim)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output
    
    def freeze (self):
        # To freeze the residual layers
        for param in self.model.parameters ():
            param.requires_grad = False

        for param in self.fc.parameters ():
            param.requires_grad = True
        return
    
    def unfreeze (self):
        # Unfreeze all layers
        for param in self.model.parameters ():
            param.requires_grad = True
        for param in self.fc.parameters ():
            param.requires_grad = True
        return

In [61]:
def compute_metrics (labels, pred_pr):
    
    preds   = pred_pr.argmax (-1)             #;print ('labels.shape=', labels.shape, 'preds.shape=', preds.shape, 'pred_logits.shape=', pred_logits.shape)
    precision, recall, f1, _ = precision_recall_fscore_support (labels, preds, average='macro')
    acc     = accuracy_score (labels, preds)
    mcc     = matthews_corrcoef (labels, preds)   # matthews correlation coefficient
    auc     = -1
    try:
        auc = roc_auc_score (labels, pred_pr[:, 1])
    except:
        pass
    metrics = {
        'mcc'      : mcc,
        'accuracy' : acc,
        'f1'       : f1,
        'precision': precision,
        'recall'   : recall,
        'auc'      : auc
    }
    return metrics

def compute_multilabel_binary_metrics (labels, logits):
    
    pred_pr = sigmoid (logits)    
    metrics = []
    n_class = labels.shape[1]
    for i in range (n_class):
        
        label  = labels[:, i]
        prob1  = pred_pr[:, i]
        prob0  = 1 - prob1
        pred_p = np.hstack ((prob0.reshape ((-1, 1)), prob1.reshape ((-1, 1))))
        scores = compute_metrics (label, pred_p)
        metrics.append (scores)
        
    # Now Avg over each classes
    metrics_df = pd.DataFrame (metrics)  
    auc = list (metrics_df['auc'].values)
    auc = np.mean ([a for a in auc if a >= 0])
    metrics_df.drop (columns=['auc'], inplace=True)
    metrics_df = metrics_df.mean ()
    metrics_df['auc'] = auc
    return metrics_df.to_dict ()

In [62]:
def getFolds ():
    
    train_folds_df = pd.read_csv (CFG.train_csv)
    label = train_folds_df[CFG.label_cols]
    if len (CFG.label_cols) > 1:
        label = train_folds_df[CFG.label_cols[0]]
        
    skf = StratifiedKFold (n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
    for n, (train_index, val_index) in enumerate (skf.split (train_folds_df, label)):
        train_folds_df.loc[val_index, 'fold'] = int (n)
    train_folds_df['fold'] = train_folds_df['fold'].astype (int)
    # print (train_folds_df.groupby (['fold', label]).size ())
        
    return train_folds_df

In [63]:
def sigmoid (x):  
    return np.exp (-np.logaddexp (0, -x))

In [64]:
class FFN (nn.Module):
    
    def __init__(self, inSize, outSize, drop_prob=0.2):
        
        super(FFN, self).__init__()
        
        self.dropout    = nn.Dropout (drop_prob)
        self.batchnorm0 = nn.BatchNorm1d (inSize)
        self.dense1     = nn.Linear (inSize, outSize)        
        return

    def forward (self, X):
        
        X = self.dense1 (self.dropout (self.batchnorm0 (X)))
        return X

In [65]:
def evaluate (validation_dataloader, models, ffn, criterion, device):
        
        t0 = time.time ()
        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        if ffn is not None:
            ffn.eval ()

        # Tracking variables 
        total_eval_mcc       = 0
        total_eval_f1        = 0
        total_eval_precision = 0
        total_eval_recall    = 0
        total_eval_auc       = 0
        total_eval_accuracy  = 0
        total_eval_loss      = 0
        nb_eval_steps        = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
                
            preds = []
            images = batch[0].to (device)                   #;print ('images.shape =', images.shape)
            for model in models:
                model.eval ()
                with torch.no_grad ():
                    pred = model (images)                   #;print ('pred.shape =', pred.shape)
                    preds.append (pred)                
            # aggregating model
            labels   = batch[1].to (device)
            features = torch.hstack (preds)                 #;print ('features.shape =', features.shape)
            if ffn is not None:
                with torch.no_grad ():
                    logits   = ffn (features)
            else:
                logits = features
            loss     = criterion (logits, labels)            
            total_eval_loss += loss.item ()
            logits   = logits.detach ().cpu ().numpy ()
            labels   = labels.detach ().cpu ().numpy ()
            
            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            metrics               = compute_multilabel_binary_metrics (labels, logits)
            total_eval_mcc       += metrics['mcc']
            total_eval_f1        += metrics['f1']
            total_eval_precision += metrics['precision']
            total_eval_recall    += metrics['recall']
            total_eval_auc       += metrics['auc']
            total_eval_accuracy  += metrics['accuracy']
        # epoch end
        
        # Report the final accuracy for this validation run.
        avg_val_f1 = total_eval_f1 / len (validation_dataloader)
        print ("  F1: {0:.3f}".format (avg_val_f1))
        avg_val_mcc = total_eval_mcc / len (validation_dataloader)
        print ("  MCC: {0:.3f}".format (avg_val_mcc))
        avg_val_precision = total_eval_precision / len (validation_dataloader)
        print ("  Precision: {0:.3f}".format (avg_val_precision))
        avg_val_recall = total_eval_recall / len (validation_dataloader)
        print ("  Recall: {0:.3f}".format (avg_val_recall))
        avg_val_auc = total_eval_auc / len (validation_dataloader)
        print ("  AUC: {0:.3f}".format (avg_val_auc))
        avg_val_accuracy = total_eval_accuracy / len (validation_dataloader)
        print ("  Accuracy: {0:.3f}".format (avg_val_accuracy))
        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len (validation_dataloader)
        
        # Measure how long the validation run took.        
        print ("  Validation Loss: {0:.2f}".format (avg_val_loss))        
        return avg_val_loss, avg_val_f1, avg_val_mcc, avg_val_auc, avg_val_precision, avg_val_recall, avg_val_accuracy

In [66]:
def one_batch_opt (labels, pred_logits, ffn, criterion, optimizer, lr_scheduler):
    
    # aggregating model
    features = torch.hstack (pred_logits)                 #;print ('features.shape =', features.shape)
    logits   = ffn (features)
    loss     = criterion (logits, labels)
    loss.backward ()
    torch.nn.utils.clip_grad_norm_ (ffn.parameters (), CFG.max_grad_norm)
    optimizer.step ()
    lr_scheduler.step ()
    return

def inference_train (models, train_dataloader, device, ffn, criterion, optimizer, lr_scheduler, epochs, valid_dataloader):
    
    global CFG
    # eval just 1 model as baseline
    metrics = evaluate (valid_dataloader, [models[0]], None, criterion, device)    
    print ('model[0] eval metrics:', metrics)
    max_auc = 0.0
    for epoch in tqdm (range (epochs)):
        print ('epoch', epoch)
        if epoch == 0:
            for b_i, batch in enumerate (train_dataloader):

                pred_logits = []
                images = batch[0].to (device)                   #;print ('images.shape =', images.shape)
                for m_i, model in enumerate (models):
                    with torch.no_grad ():
                        pred_logit = model (images)                   #;print ('pred_logit.shape =', pred_logit.shape)
                    CFG.MODEL_LOGITS[m_i].append (pred_logit)
                    pred_logits.append (pred_logit)                
                
                labels = batch[1]
                CFG.BATCH_LABELS.append (labels)
                labels = labels.to (device)
                one_batch_opt (labels, pred_logits, ffn, criterion, optimizer, lr_scheduler)
        else:
            
            for b_i in range (len (train_dataloader)):
                
                pred_logits = []                
                for m_i, model in enumerate (models):
                    with torch.no_grad ():
                        pred_logit = CFG.MODEL_LOGITS[m_i][b_i]          #;print ('pred_logit.shape =', pred_logit.shape)
                        pred_logits.append (pred_logit)                

                labels = CFG.BATCH_LABELS[b_i].to (device)
                one_batch_opt (labels, pred_logits, ffn, criterion, optimizer, lr_scheduler)
            
        # get metrics
        avg_val_loss, avg_val_f1, avg_val_mcc, avg_val_auc, avg_val_precision, avg_val_recall, avg_val_accuracy = evaluate (valid_dataloader, models, ffn, criterion, device)
        print ('Epoch -', epoch)
        # print (avg_val_loss, avg_val_f1, avg_val_mcc, avg_val_auc, avg_val_precision, avg_val_recall, avg_val_accuracy)
        if avg_val_auc > max_auc:
            print ("Saving Max-AUC !!")
            max_auc = avg_val_auc
            torch.save (ffn.state_dict (), "ffn_auc.pth")
    return

In [67]:
def train_fold_loop (fold, train_df=getFolds ()):
    
    print (f"========== fold: {fold} training ==========")
    global CFG
    CFG.MODELS     = []
    trn_idx        = train_df[train_df['fold'] != fold].index
    val_idx        = train_df[train_df['fold'] == fold].index
    train_folds_df = train_df.loc[trn_idx].reset_index (drop=True)
    valid_folds_df = train_df.loc[val_idx].reset_index (drop=True)
    train_dataset  = ImgDataset (train_folds_df, transform=get_transforms (), label_cols=CFG.label_cols, img_dir=CFG.train_path)
    valid_dataset  = ImgDataset (valid_folds_df, transform=get_transforms (), label_cols=CFG.label_cols, img_dir=CFG.train_path)
    train_dataloader = DataLoader (train_dataset, shuffle=False, batch_size=CFG.batch_size, pin_memory=True, num_workers=4)
    valid_dataloader = DataLoader (valid_dataset, shuffle=False, batch_size=CFG.batch_size, num_workers=4)
    class_wt       = 1 / np.array (train_df[CFG.label_cols].sum (axis=0).values)
    class_wt       = torch.tensor (class_wt / np.sum (class_wt))   ;print ('class_wt =', class_wt)
    class_wt       = class_wt.to (CFG.device)
    del train_df; gc.collect ()
    
    for path in CFG.MODEL_PATHS:
        model = ResNet200D ().eval ()
        model.load_state_dict (torch.load (path, map_location=torch.device ('cpu'))['model'])
        model.freeze ()
        CFG.MODELS.append (model.to (CFG.device))
        CFG.MODEL_LOGITS.append ([])
    
    ffn          = FFN ((len (CFG.MODELS))*11, 11, 0.2)
    # ffn.load_state_dict (torch.load (CFG.ffn_path, map_location=torch.device ('cpu')))
    ffn          = ffn.train ().to (CFG.device)
    criterion    = nn.BCEWithLogitsLoss (pos_weight=class_wt)
    optimizer    = AdamW (ffn.parameters (), lr=CFG.lr, eps=1e-8)
    epochs       = CFG.epochsNx
    lr_scheduler = get_linear_schedule_with_warmup (optimizer, num_warmup_steps=500, num_training_steps=len (train_dataloader)*epochs)
    inference_train (CFG.MODELS, train_dataloader, CFG.device, ffn, criterion, optimizer, lr_scheduler, epochs, valid_dataloader)
    torch.save (ffn.state_dict (), 'ffn.pth')
    return ffn

In [68]:
train_fold_loop (0)

========== fold: 0 training ==========
class_wt = tensor([0.5914, 0.0411, 0.0065, 0.1674, 0.0883, 0.0170, 0.0097, 0.0146, 0.0055,
        0.0022, 0.0563], dtype=torch.float64)


  0%|          | 0/1 [00:00<?, ?it/s]

  F1: 0.939
  MCC: 0.000
  Precision: 0.955
  Recall: 0.932
  AUC: nan
  Accuracy: 0.955
  Validation Loss: 0.16
model[0] eval metrics: (0.1638001650571823, 0.9393939393939394, 0.0, nan, 0.9545454545454546, 0.9318181818181818, 0.9545454545454546)
epoch 0


100%|██████████| 1/1 [00:41<00:00, 41.99s/it]

  F1: 0.455
  MCC: 0.000
  Precision: 0.455
  Recall: 0.455
  AUC: nan
  Accuracy: 0.455
  Validation Loss: 1.85
Epoch - 0
1.8471516370773315 0.45454545454545453 0.0 nan 0.45454545454545453 0.45454545454545453 0.45454545454545453


FFN(
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm0): BatchNorm1d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=44, out_features=11, bias=True)
)

# Inference

In [69]:
def inference (models, ffn, test_loader, device):
    
    tk0 = tqdm (enumerate (test_loader), total=len (test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to (device)
        all_preds_1 = []
        all_preds_2 = []
        for model in models:
            with torch.no_grad ():
                y_preds1 = model (images)
                y_preds2 = model (images.flip (-1))            
            all_preds_1.append (y_preds1)
            all_preds_2.append (y_preds2)
            
        all_preds_1 = torch.hstack (all_preds_1)
        all_preds_2 = torch.hstack (all_preds_2)
        all_preds_1 = torch.sigmoid (ffn (all_preds_1))
        all_preds_2 = torch.sigmoid (ffn (all_preds_2))
        all_preds   = ((all_preds_1 + all_preds_2) / 2).detach().cpu ().numpy ()
        probs.append (all_preds)
    probs = np.concatenate (probs)
    return probs

In [70]:
def get_pred ():
    
    global CFG    
    CFG.MODELS = []
    for path in CFG.MODEL_PATHS:
            model = ResNet200D ().eval ()
            model.load_state_dict (torch.load (path, map_location=torch.device ('cpu'))['model'])
            model.freeze ()
            CFG.MODELS.append (model.to (CFG.device))

    ffn = FFN ((len (CFG.MODELS))*11, 11, 0.2)
    ffn.load_state_dict (torch.load (CFG.ffn_path, map_location=torch.device ('cpu')))
    ffn = ffn.eval ().to (CFG.device)
    torch.save (ffn.state_dict (), "ffn_end.pth")

    test_dataset = ImgDataset (test_df, transform=get_transforms (), label_cols=[], img_dir=CFG.test_path)
    test_loader  = DataLoader (test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4 , pin_memory=True)
    predictions  = inference (CFG.MODELS, ffn, test_loader, CFG.device)
    return predictions

In [71]:
# target_cols = test_df.iloc[:, 1:12].columns.tolist ()
# predictions = get_pred ()
# test_df[target_cols] = predictions
# test_df[['StudyInstanceUID'] + target_cols].to_csv ('submission.csv', index=False)
# test_df.head ()